In [ ]:
import pandas as pd
import openpyxl
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize